# Predicting the Success of a Reddit Submission with Deep Learning and Keras

by Max Woolf

*Max's open-source projects are supported by his [Patreon](https://www.patreon.com/minimaxir). If you found this project helpful, any monetary contributions to the Patreon are appreciated and will be put to good creative use.*



BigQuery used to get data:

```sql
#standardSQL 
SELECT id, title,
  CAST(FORMAT_TIMESTAMP('%H', TIMESTAMP_SECONDS(created_utc), 'America/New_York') AS INT64) AS hour,
  CAST(FORMAT_TIMESTAMP('%M', TIMESTAMP_SECONDS(created_utc), 'America/New_York') AS INT64) AS minute,
  CAST(FORMAT_TIMESTAMP('%w', TIMESTAMP_SECONDS(created_utc), 'America/New_York') AS INT64) AS dayofweek,
  CAST(FORMAT_TIMESTAMP('%j', TIMESTAMP_SECONDS(created_utc), 'America/New_York') AS INT64) AS dayofyear,
  IF(PERCENT_RANK() OVER (ORDER BY score ASC) >= 0.50, 1, 0) as is_top_submission
  FROM `fh-bigquery.reddit_posts.*`
  WHERE (_TABLE_SUFFIX BETWEEN '2017_01' AND '2017_04')
  AND subreddit = 'AskReddit'
```

# Setup

In [1]:
import numpy as np
import os
import csv
from random import random, sample, seed

data_path = '/Volumes/Extreme 510/Data/askreddit_data_timings.csv'
embeddings_path = '/Volumes/Extreme 510/Data/glove.6B.50d.txt'

In [2]:
titles = []
hours = []
minutes = []
dayofweeks = []
dayofyears = []
is_top_submission = []

with open(data_path, 'r', encoding="latin1") as f:
    reader = csv.DictReader(f)
    for submission in reader:
        titles.append(submission['title'])
        hours.append(submission['hour'])
        minutes.append(submission['minute'])
        dayofweeks.append(submission['dayofweek'])
        dayofyears.append(submission['dayofyear'])
        is_top_submission.append(submission['is_top_submission'])
            
titles = np.array(titles)
hours = np.array(hours, dtype=int)
minutes = np.array(minutes, dtype=int)
dayofweeks = np.array(dayofweeks, dtype=int)
dayofyears = np.array(dayofyears, dtype=int)
is_top_submission = np.array(is_top_submission, dtype=int)

In [3]:
print(titles[0:2])
print(titles.shape)
print(hours[0:2])
print(minutes[0:2])
print(dayofweeks[0:2])
print(dayofyears[0:2])
print(is_top_submission[0:2])

['People who have been cheated on: how did you find out?'
 "What is the biggest display of confidence/charisma you've ever seen?"]
(976538,)
[ 1 16]
[10 48]
[3 3]
[46 46]
[0 0]


In [4]:
1 - np.mean(is_top_submission)

0.64075949937432031

The No-Information Rate is 64% (i.e. simply say all AskReddit submissions are terrible), so any model trained must do better than that.

# Process /r/AskReddit Submission Title Text

In [5]:
from keras.preprocessing import sequence
from keras.preprocessing.text import text_to_word_sequence, Tokenizer

max_features = 40000

word_tokenizer = Tokenizer(max_features)
word_tokenizer.fit_on_texts(titles)

print(str(word_tokenizer.word_counts)[0:100])
print(str(word_tokenizer.word_index)[0:100])
print(len(word_tokenizer.word_counts))   # true word count

Using TensorFlow backend.


{'people': 77432, 'who': 80438, 'have': 120730, 'been': 24255, 'cheated': 946, 'on': 87077, 'how': 1
{'you': 1, 'what': 2, 'the': 3, 'to': 4, 'a': 5, 'of': 6, 'your': 7, 'is': 8, 'do': 9, 'and': 10, 'i
98770


In [6]:
titles_tf = word_tokenizer.texts_to_sequences(titles)

print(titles_tf[0])

[27, 26, 17, 75, 1222, 24, 15, 33, 1, 119, 55]


In [7]:
maxlen = 20
titles_tf = sequence.pad_sequences(titles_tf, maxlen=maxlen)

print(titles_tf[0])

[   0    0    0    0    0    0    0    0    0   27   26   17   75 1222   24
   15   33    1  119   55]


## Add Pretrained Embeddings

Adapted from [the official keras tutorial](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html).

Use pretrained GloVe embeddings to both give Embeddings training a good start, and to account for words that might be present in the test set but not in the training set.

First, load the 50D embeddings into memory.

In [8]:
embedding_vectors = {}

with open(embeddings_path, 'r') as f:
    for line in f:
        line_split = line.strip().split(" ")
        vec = np.array(line_split[1:], dtype=float)
        word = line_split[0]
        embedding_vectors[word] = vec
        
print(embedding_vectors['you'])

[ -1.09190000e-03   3.33240000e-01   3.57430000e-01  -5.40410000e-01
   8.20320000e-01  -4.93910000e-01  -3.25880000e-01   1.99720000e-03
  -2.38290000e-01   3.55540000e-01  -6.06550000e-01   9.89320000e-01
  -2.17860000e-01   1.12360000e-01   1.14940000e+00   7.32840000e-01
   5.11820000e-01   2.92870000e-01   2.83880000e-01  -1.35900000e+00
  -3.79510000e-01   5.09430000e-01   7.07100000e-01   6.29410000e-01
   1.05340000e+00  -2.17560000e+00  -1.32040000e+00   4.00010000e-01
   1.57410000e+00  -1.66000000e+00   3.77210000e+00   8.69490000e-01
  -8.04390000e-01   1.83900000e-01  -3.43320000e-01   1.07140000e-02
   2.39690000e-01   6.67480000e-02   7.01170000e-01  -7.37020000e-01
   2.08770000e-01   1.15640000e-01  -1.51900000e-01   8.59080000e-01
   2.26200000e-01   1.65190000e-01   3.63090000e-01  -4.56970000e-01
  -4.89690000e-02   1.13160000e+00]


Initialize the weights matrix as zeroes, then replace the corresponding index of the weights matrix with the index of the corresponding word.

In [9]:
weights_matrix = np.zeros((max_features + 1, 50))

for word, i in word_tokenizer.word_index.items():

    embedding_vector = embedding_vectors.get(word)
    if embedding_vector is not None and i <= max_features:
        weights_matrix[i] = embedding_vector

# index 0 vector should be all zeroes, index 1 vector should be the same one as above
print(weights_matrix[0:2,:])

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [ -1.09190000e-03   3.33240000e-01   3.57430000e-01  -5.40410000e-01
    8.20320000e-01  -4.93910000e-01  -3.25880000e-01 

# Process Other Metadata

All metadata must be zero-indexed integers.

* `hours` in the correct format. (`0` = 12AM EST, `23` = 11PM EST)
* `dayofweeks` is in the correct format (`0` = Sunday, `6` = Saturday)
* `minutes` is in the correct format verbatim.
* `dayofyears` is 1-indexed, so must subtract 1.

In [10]:
dayofyears_tf = dayofyears - 1

print(dayofyears_tf[0:10])

[ 45  45  44 104  47  46 115 115  57   1]


# Build the Model

Use Keras's functional API to build a branching model.

In [11]:
from keras.models import Input, Model
from keras.layers import Dense, Embedding, GlobalAveragePooling1D, concatenate, Activation
from keras.layers.core import Masking, Dropout, Reshape
from keras.layers.normalization import BatchNormalization

batch_size = 32
embedding_dims = 50
epochs = 20

## Text Branch

Encode the text using a mock fasttext approach. Use `weights_matrix` derived above.

In [12]:
titles_input = Input(shape=(maxlen,), name='titles_input')
titles_embedding = Embedding(max_features + 1, embedding_dims, weights=[weights_matrix])(titles_input)
titles_pooling = GlobalAveragePooling1D()(titles_embedding)

Add an auxillary output to regularize the text component.

In [13]:
aux_output = Dense(1, activation='sigmoid', name='aux_out')(titles_pooling)

## Metadata Branch

Each metadata variable gets its own input and Embeddings. (size of each Embedding is already known by construction of the variables)

`Reshape` is necessary to convert from 2D to 1D.

In [14]:
meta_embedding_dims = 64

hours_input = Input(shape=(1,), name='hours_input')
hours_embedding = Embedding(24, meta_embedding_dims)(hours_input)
hours_reshape = Reshape((meta_embedding_dims,))(hours_embedding)

dayofweeks_input = Input(shape=(1,), name='dayofweeks_input')
dayofweeks_embedding = Embedding(7, meta_embedding_dims)(dayofweeks_input)
dayofweeks_reshape = Reshape((meta_embedding_dims,))(dayofweeks_embedding)

minutes_input = Input(shape=(1,), name='minutes_input')
minutes_embedding = Embedding(60, meta_embedding_dims)(minutes_input)
minutes_reshape = Reshape((meta_embedding_dims,))(minutes_embedding)

dayofyears_input = Input(shape=(1,), name='dayofyears_input')
dayofyears_embedding = Embedding(366, meta_embedding_dims)(dayofyears_input)
dayofyears_reshape = Reshape((meta_embedding_dims,))(dayofyears_embedding)

Minutes and dayofyears are single scalars; no need to `Reshape`.

## Merge the Branches and Complete Model

Combine the 4 embeddings (200D total), add a two-layer MLP to understand latent characteristic, output 1 value for logistic regression.

In [15]:
merged = concatenate([titles_pooling, hours_reshape, dayofweeks_reshape, minutes_reshape, dayofyears_reshape])

hidden_1 = Dense(256, activation='relu')(merged)
hidden_1 = BatchNormalization()(hidden_1)

main_output = Dense(1, activation='sigmoid', name='main_out')(hidden_1)

## Compile the Model

In [16]:
model = Model(inputs=[titles_input,
                      hours_input,
                      dayofweeks_input,
                      minutes_input,
                      dayofyears_input], outputs=[main_output, aux_output])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              loss_weights=[1, 0.2])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
titles_input (InputLayer)        (None, 20)            0                                            
____________________________________________________________________________________________________
hours_input (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
dayofweeks_input (InputLayer)    (None, 1)             0                                            
____________________________________________________________________________________________________
minutes_input (InputLayer)       (None, 1)             0                                            
___________________________________________________________________________________________

In [17]:
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

plot_model(model, to_file='model.png')
plot_model(model, to_file='model_shapes.png', show_shapes=True)

![](model.png)

# Train the Model!

Randomize the model before training, since Keras [takes the last 20%](https://keras.io/getting-started/faq/#how-is-the-validation-split-computed) as the validation set.

In [18]:
seed(123)
split = 0.2

# returns randomized indices with no repeats
idx = sample(range(titles_tf.shape[0]), titles_tf.shape[0])

titles_tf = titles_tf[idx, :]
hours = hours[idx]
dayofweeks = dayofweeks[idx]
minutes = minutes[idx]
dayofyears_tf = dayofyears_tf[idx]
is_top_submission = is_top_submission[idx]

Determine No-Information Rate of the test set: the `val_main_out_acc` must be better than it.

In [19]:
print(1 - np.mean(is_top_submission[:(int(titles_tf.shape[0] * split))]))

0.64137998126


Log results to file:

In [20]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('training.csv')

Train the model (finally!)

In [21]:
model.fit([titles_tf, hours, dayofweeks, minutes, dayofyears_tf], [is_top_submission, is_top_submission],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=split, callbacks=[csv_logger])

Train on 781230 samples, validate on 195308 samples
Epoch 1/20
781230/781230 [==============================] - 650s - loss: 0.6932 - main_out_loss: 0.5753 - aux_out_loss: 0.5894 - main_out_acc: 0.6590 - aux_out_acc: 0.6524 - val_loss: 0.6799 - val_main_out_loss: 0.5635 - val_aux_out_loss: 0.5819 - val_main_out_acc: 0.6682 - val_aux_out_acc: 0.6558
Epoch 2/20
781230/781230 [==============================] - 652s - loss: 0.6716 - main_out_loss: 0.5569 - aux_out_loss: 0.5736 - main_out_acc: 0.6737 - aux_out_acc: 0.6658 - val_loss: 0.6778 - val_main_out_loss: 0.5615 - val_aux_out_loss: 0.5816 - val_main_out_acc: 0.6698 - val_aux_out_acc: 0.6611
Epoch 3/20
781230/781230 [==============================] - 736s - loss: 0.6645 - main_out_loss: 0.5505 - aux_out_loss: 0.5700 - main_out_acc: 0.6799 - aux_out_acc: 0.6697 - val_loss: 0.6782 - val_main_out_loss: 0.5617 - val_aux_out_loss: 0.5822 - val_main_out_acc: 0.6692 - val_aux_out_acc: 0.6611
Epoch 4/20
781230/781230 [=========================

# Using the Model to Make Predictions

Use the created model to optimize a submission to /r/AskReddit.

Example submission with only 4 upvotes: [Which movie's plot would drastically change if you removed a letter from its title?](https://www.reddit.com/r/AskReddit/comments/5odcpd/which_movies_plot_would_drastically_change_if_you/)

## Optimize Text (fixing Time Submitted)

First create a function to include arbitrary text as model-acceptable format. (using dictionary created from the Tokenizer)

In [22]:
def encode_text(text, maxlen):
    encoded = word_tokenizer.texts_to_sequences([text])
    return sequence.pad_sequences(encoded, maxlen=maxlen)

In [36]:
input_text = "Which movie's plot would drastically change if you removed a letter from its title?"
encoded_text = encode_text(input_text, maxlen)
print(encoded_text)

[[   0    0    0    0    0    0   68 6380  923   18 3403  142   23    1
  1513    5 1146   44  302  508]]


Identify a baseline using the real values.

The first value (index 0) is the value from the `main_output`, the second is from the fasttext `aux_output`.

In [37]:
input_hour = np.array([15])
input_minute = np.array([46])
input_dayofweek = np.array([1])
input_dayofyear = np.array([16 - 1])

model.predict([encoded_text, input_hour, input_dayofweek, input_minute, input_dayofyear])

[array([[ 0.66917652]], dtype=float32), array([[ 0.40641001]], dtype=float32)]

So can see if we can to perform better than 0.67 probability. Make minor changes without avertly changing meaning.

In [38]:
input_text = "Which movie's plot would change if you removed a letter from its title?"
encoded_text = encode_text(input_text, maxlen)
model.predict([encoded_text, input_hour, input_dayofweek, input_minute, input_dayofyear])

[array([[ 0.68286121]], dtype=float32), array([[ 0.40531182]], dtype=float32)]

Removing "drastically" makes things better.

How about fixing the tense of "removed"?

In [41]:
input_text = "Which movie's plot would change if you remove a letter from its title?"
encoded_text = encode_text(input_text, maxlen)
model.predict([encoded_text, input_hour, input_dayofweek, input_minute, input_dayofyear])

[array([[ 0.69150311]], dtype=float32), array([[ 0.4588666]], dtype=float32)]

Probability increased. From that headline, how about changing "Which" to "What?"

In [42]:
input_text = "What movie's plot would change if you remove a letter from its title?"
encoded_text = encode_text(input_text, maxlen)
model.predict([encoded_text, input_hour, input_dayofweek, input_minute, input_dayofyear])

[array([[ 0.7321561]], dtype=float32), array([[ 0.51213974]], dtype=float32)]

What about using more emphasis: add "single" to letter.

In [44]:
input_text = "What movie's plot would change if you remove a single letter from its title?"
encoded_text = encode_text(input_text, maxlen)
model.predict([encoded_text, input_hour, input_dayofweek, input_minute, input_dayofyear])

[array([[ 0.75322324]], dtype=float32), array([[ 0.55223811]], dtype=float32)]

## Optimize Time Submitted (fixing Text)

Do a hyperparameter grid search for all combinations of `hour`, `minute`, and `dayofweek` (`dayofyear` must be offset accordingly with `dayofweek`) and record the probabilities.

In [45]:
results_array = []

for input_dayofweek in range(6):
    for input_hour in range(23):
        for input_minute in range(59):
            results_array.append([input_dayofweek,
                                  input_hour,
                                  input_minute,
                                  model.predict([encoded_text,
                                                 np.array([input_hour]),
                                                 np.array([input_dayofweek]),
                                                 np.array([input_minute]),
                                                 input_dayofyear + input_dayofweek - 1])[0]])
            
results_array = np.array(results_array)

In [47]:
print(results_array[results_array[:,3].argsort()[::-1]][0:20,:])

[[  0.          22.          55.           0.84148395]
 [  0.          22.          13.           0.83928609]
 [  0.          22.           0.           0.83868033]
 [  0.          22.          49.           0.83778793]
 [  0.          22.          27.           0.83483583]
 [  0.          22.          19.           0.83461344]
 [  0.          22.          16.           0.83399975]
 [  0.          22.          39.           0.83264273]
 [  0.          22.          47.           0.82798171]
 [  0.          22.          33.           0.82755792]
 [  0.          22.          43.           0.82691711]
 [  0.          22.          42.           0.82675755]
 [  0.          22.          58.           0.82663274]
 [  0.          22.          10.           0.82661295]
 [  0.          22.          37.           0.82582957]
 [  0.          22.          25.           0.82406569]
 [  0.          22.          12.           0.82374126]
 [  0.          21.          13.           0.82372391]
 [  2.    

# Extract Embeddings

Add the embeddings for each Embeddings layer into a .txt file. (skipping word vectors, since the resulting data file will be huge.

Create a helper function to do that:

In [31]:
def write_embeddings(weights, file_name):
    f = open(file_name + ".txt", 'w')
    for i in range(weights.shape[0]):
            f.write(str(i) + " " + " ".join(str(x) for x in weights[i]) + "\n")
    f.close()

The layers are in the same order as they were instantiated (can look at summary to get index).

In [32]:
write_embeddings(model.layers[6].get_weights()[0], "hours_embeddings")
write_embeddings(model.layers[7].get_weights()[0], "dayofweeks_embeddings")
write_embeddings(model.layers[8].get_weights()[0], "minutes_embeddings")

# remember to add +1 back to index when doing analysis
# additionally, the last value corresponds to i = 366
write_embeddings(model.layers[9].get_weights()[0], "dayofyears_embeddings")

# LICENSE

The MIT License (MIT)

Copyright (c) 2017 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.